# Men from Mars, Women from Venus: both can code .NET in Jupyter

@rondagdag

### .NET Interactive https://github.com/dotnet/interactive

### To install in raspberry pi
```
curl -L https://raw.githubusercontent.com/colombod/pi-top/master/setup-device.sh | bash -e
source .jupyter_venv/bin/activate
jupyter lab --no-browser

```

Rasperry Pi Sense Hat installation instructions
https://www.raspberrypi.org/documentation/hardware/sense-hat/

In [1]:
#!lsmagic

The below script needs to be able to find the current output cell; this is an easy method to get it.

.NET kernel #!csharp , #!c# , #!C# Run the code that follows using the csharp kernel. #!fsharp , #!f# , #!F# Run the code that follows using the fsharp kernel. #!pwsh , #!powershell Run the code that follows using the pwsh kernel. #!javascript , #!js Run the code that follows using the javascript kernel. #!html Run the code that follows using the html kernel. #!value Stores a value #!lsmagic List the available magic commands / directives #!markdown Convert the code that follows from Markdown into HTML #!time Time the execution of the following code in the submission. #!log Enables session logging. #!about Show version and build information #!connect Connects additional subkernels named-pipe Connects to a kernel using named pipes signalr Connects to a kernel using SignalR

csharp kernel #i <null> #r <null> #!whos , #!who Display the names of the current top-level variables and their values. #!share Share a .NET variable between subkernels #!lsmagic List the available magic commands / directives

fsharp kernel #i <null> #r <null> #!whos , #!who Display the names of the current top-level variables and their values. #!share Share a .NET variable between subkernels #!lsmagic List the available magic commands / directives

pwsh kernel #!share Share a .NET variable between subkernels #!lsmagic List the available magic commands / directives

javascript kernel #!lsmagic List the available magic commands / directives

html kernel #!lsmagic List the available magic commands / directives

value kernel #!lsmagic List the available magic commands / directives

In [2]:
#r "nuget:System.Device.Gpio,1.1.0-prerelease.20276.1"
#r "nuget:IoT.Device.Bindings,1.1.0-prerelease.20276.1"
#r "nuget:UnitsNet,4.68.0"

Installed package System.Device.Gpio version 1.1.0-prerelease.20276.1

Installed package IoT.Device.Bindings version 1.1.0-prerelease.20276.1

Installed package UnitsNet version 4.68.0

### Display the html

In [3]:
var someHtml = HTML("<b style=\"color:blue\">Sensor data!</b>");
var tempOutput = display(someHtml);

103.01375045776368

In [4]:
using Iot.Device.SenseHat;
using System;
var temperatures = new List<(int idx, double val)>();

async Task readTemperature(int count = 10){   
    using (var sh = new SenseHat())
    {
        var initcount = temperatures.Count;
        for (var i = initcount; i < initcount + count; i++){        
            temperatures.Add((i, sh.Temperature2.DegreesFahrenheit));
            tempOutput.Update(HTML($"<b style=\"color:green\">{sh.Temperature2.DegreesFahrenheit}</b>"));
            await Task.Delay(500);
        }
    }
}

### Read Temperatures

In [5]:
await readTemperature(20);

In [6]:
using XPlot.Plotly;

In [7]:
var values = temperatures.Select(i => i.val);
values

index,value
0,102.94625320434571
1,102.89375228881836
2,102.90499954223633
3,102.91249771118164
4,102.93874816894531
5,102.95000228881835
6,102.96499862670899
7,102.9724967956543
8,102.9724967956543
9,102.9724967956543


In [8]:
var openSeries = new Graph.Scatter
{
    name = "Open",
    x = temperatures.Select(i => i.idx),
    y = values
};


var chart = Chart.Plot(new[] {openSeries});
chart.WithTitle("Open");

### Display the chart

In [9]:
display(chart);

In [10]:
async IAsyncEnumerable<double> ReadTemperatureAsync(int count = 10){   
    using (var sh = new SenseHat())
    {
        for (var i = 0; i < count; i++){
            await Task.Delay(1000);
            yield return (sh.Temperature2.DegreesFahrenheit);
        }
    };
}

In [11]:
#r "nuget:Microsoft.Data.Analysis"
using Microsoft.Data.Analysis;

Installed package Microsoft.Data.Analysis version 0.4.0

In [12]:
var output = display("Counting...");
PrimitiveDataFrameColumn<DateTime> dateTimes = new PrimitiveDataFrameColumn<DateTime>("DateTimes"); // Default length is 0.
PrimitiveDataFrameColumn<double> fahrenheit = new PrimitiveDataFrameColumn<double>("Fahrenheit"); // Makes a column of length 3. Filled with nulls initially
await foreach (var result in ReadTemperatureAsync(15))
{
   output.Update(result);
   dateTimes.Append(DateTime.Now);
   fahrenheit.Append(result);
}

103.09250183105469

### create a new Dataframe with 2 columns

In [13]:
DataFrame df =  new DataFrame(dateTimes, fahrenheit);

In [14]:
df

Columns,Rows
"[ [ 2020-08-06 20:16:40Z, 2020-08-06 20:16:41Z, 2020-08-06 20:16:42Z, 2020-08-06 20:16:43Z, 2020-08-06 20:16:44Z, 2020-08-06 20:16:45Z, 2020-08-06 20:16:46Z, 2020-08-06 20:16:47Z, 2020-08-06 20:16:48Z, 2020-08-06 20:16:49Z, 2020-08-06 20:16:50Z, 2020-08-06 20:16:51Z, 2020-08-06 20:16:52Z, 2020-08-06 20:16:53Z, 2020-08-06 20:16:54Z ], [ 103.24249954223633, 103.20125274658203, 103.1412467956543, 103.08124771118165, 103.0512481689453, 103.12625045776367, 103.15250091552734, 103.15625, 103.12999954223633, 103.09250183105469, 103.14875183105468, 103.15625, 103.1974967956543, 103.12625045776367, 103.09250183105469 ] ]","[ [ 2020-08-06 20:16:40Z, 103.24249954223633 ], [ 2020-08-06 20:16:41Z, 103.20125274658203 ], [ 2020-08-06 20:16:42Z, 103.1412467956543 ], [ 2020-08-06 20:16:43Z, 103.08124771118165 ], [ 2020-08-06 20:16:44Z, 103.0512481689453 ], [ 2020-08-06 20:16:45Z, 103.12625045776367 ], [ 2020-08-06 20:16:46Z, 103.15250091552734 ], [ 2020-08-06 20:16:47Z, 103.15625 ], [ 2020-08-06 20:16:48Z, 103.12999954223633 ], [ 2020-08-06 20:16:49Z, 103.09250183105469 ], [ 2020-08-06 20:16:50Z, 103.14875183105468 ], [ 2020-08-06 20:16:51Z, 103.15625 ], [ 2020-08-06 20:16:52Z, 103.1974967956543 ], [ 2020-08-06 20:16:53Z, 103.12625045776367 ], [ 2020-08-06 20:16:54Z, 103.09250183105469 ] ]"


In [15]:
using Microsoft.AspNetCore.Html;
Formatter<DataFrame>.Register((df, writer) =>
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent) th(c.Name)));
    var rows = new List<List<IHtmlContent>>();
    var take = 5;
    for (var i = 0; i < Math.Min(take, df.Rows.Count); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(i));
        foreach (var obj in df.Rows[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }
    
    var t = table(
        thead(
            headers),
        tbody(
            rows.Select(
                r => tr(r))));
    
    writer.Write(t);
}, "text/html");

In [16]:
df

0,2020-08-06 20:16:40Z,103.24249954223633
1,2020-08-06 20:16:41Z,103.20125274658203
2,2020-08-06 20:16:42Z,103.1412467956543
3,2020-08-06 20:16:43Z,103.08124771118165
4,2020-08-06 20:16:44Z,103.0512481689453


In [17]:
var a = df.Rows.Take(2);
a

index,value
0,"[ 2020-08-06 20:16:40Z, 103.24249954223633 ]"
1,"[ 2020-08-06 20:16:41Z, 103.20125274658203 ]"


### Count the number of rows

In [18]:
var total = df.Rows.Count();
total

15

In [19]:
PrimitiveDataFrameColumn<double> celcius = new PrimitiveDataFrameColumn<double>("Celcius", total);
var newDf = df.Clone();
newDf.Columns.Insert(2, celcius);
newDf

0,2020-08-06 20:16:40Z,103.24249954223633,<null>
1,2020-08-06 20:16:41Z,103.20125274658203,<null>
2,2020-08-06 20:16:42Z,103.1412467956543,<null>
3,2020-08-06 20:16:43Z,103.08124771118165,<null>
4,2020-08-06 20:16:44Z,103.0512481689453,<null>


In [20]:
var info = newDf.Info();
info

0,DataType,System.DateTime,System.Double,System.Double
1,Length (excluding null values),15,15,0


In [21]:
celcius.FillNulls(0, inPlace: true);
newDf

0,2020-08-06 20:16:40Z,103.24249954223633,0
1,2020-08-06 20:16:41Z,103.20125274658203,0
2,2020-08-06 20:16:42Z,103.1412467956543,0
3,2020-08-06 20:16:43Z,103.08124771118165,0
4,2020-08-06 20:16:44Z,103.0512481689453,0


In [22]:
DataFrameRow row0 = newDf.Rows[0];
row0

index,type,value
0,System.DateTime,2020-08-06 20:16:40Z
1,System.Double,103.24249954223633
2,System.Double,0


### Convert Fahrenheit and fill the Celcius

In [32]:
newDf.Columns["Celcius"] = (fahrenheit - 32) * 5/9; 
newDf

0,2020-08-06 20:16:40Z,103.24249954223633,39.579166412353516
1,2020-08-06 20:16:41Z,103.20125274658203,39.556251525878906
2,2020-08-06 20:16:42Z,103.1412467956543,39.52291488647461
3,2020-08-06 20:16:43Z,103.08124771118165,39.48958206176758
4,2020-08-06 20:16:44Z,103.0512481689453,39.47291564941406


In [33]:
var celciusMean = newDf.Columns["Celcius"].Mean();
celciusMean

39.5220832824707

In [34]:
Chart.Plot(
    new Graph.Scatter()
    {
        x = newDf.Columns["DateTimes"],
        y = newDf.Columns["Celcius"]
    }
)

In [35]:
var tempData = new List<double>();

async Task readTempData(int count = 10){   
    using (var sh = new SenseHat())
    {
        for (var i = 0; i < count; i++){        
            if(tempData.Count > 10){
                tempData.RemoveAt(0);
            }
            tempData.Add(sh.Temperature2.DegreesFahrenheit);
            Console.WriteLine(sh.Temperature2.DegreesFahrenheit);
            await Task.Delay(500);
        }
    }
}

In [49]:
#!js 

if (typeof (notebookScope.interval) !== 'undefined') {
    clearInterval(notebookScope.interval);
}

notebookScope.plot = (sgvSelector, variableName, min, max) => {   
    let dtree_require = require.config({
        paths: {
            d3: "https://d3js.org/d3.v5.min"
        }
    });
    dtree_require(["d3"], function (d3) {       
        let svg = d3.select(sgvSelector);
        svg.selectAll("defs").remove();
        svg.selectAll("g").remove();
        
        var margin = {top: 0, right: 30, bottom: 30, left: 40},
            width = 260 - margin.left - margin.right,
            height = 200 - margin.top - margin.bottom;
        
        let container = d3
            .select(sgvSelector);
        
         var width = 260 
            scaleFactor = 20, 
            barHeight = 30;
         
         var graph = d3.select("body")
            .append("svg")
                .attr("width", width + margin.left + margin.right)
                .attr("height", height + margin.top + margin.bottom)
            .append("g")
                .attr("transform",
                      "translate(" + margin.left + "," + margin.top + ")");
        
        // Add X axis
          var x = d3.scaleLinear()
            .domain([0, 10])
            .range([ 0, width ]);
        
          svg.append("g")
            .attr("transform", "translate(" + margin.left + "," + height + ")")
            .call(d3.axisBottom(x));

      // Add Y axis
          var y = d3.scaleLinear()
            .domain([102, 104])
            .range([ height, 0]);
        
          svg.append("g")
            .attr("transform", "translate(" + margin.left + ", 0)")
            .call(d3.axisLeft(y));

        updateD3Rendering = () => interactive.csharp.getVariable(variableName)
            .then(data => {
                console.log(data);
                container
                    .selectAll(".points")
                    .data(data, (d, i) => i)
                    .join(
                        enter => enter.append("circle")
                            .attr("cx", (d, i) => { console.log("i:" + i); return margin.left + x(i);} )
                            .attr("cy", function (d) { console.log("d:" + d); return y(d); } )
                            .attr("r", 2.5)
                            .style("fill", "#69b3a2")
                            .transition()
                            .duration(2000)                            
                            .attr("r", 0 ).remove());
            });
        notebookScope.interval = setInterval(() => updateD3Rendering(), 1000);
    });
}

In [57]:
await readTempData(5);

102.7474967956543
102.75125274658203
102.75125274658203
102.73624954223632
102.7212532043457


### Plot and call javascript 

In [58]:
#!html
<svg id="dataPlot1" width=500 height=300>
</svg>

#!js
notebookScope.plot("svg#dataPlot1", "tempData",102,103);

In [59]:
await readTempData(50);

102.8337532043457
102.62749862670898
102.63124771118164
102.6349967956543
102.62749862670898
102.64250183105469
102.65374908447265
102.65
102.65749816894531
102.63124771118164
102.59749908447266
102.61250228881836
102.62374954223633
102.63875274658203
102.65374908447265
102.65
102.6349967956543
102.62000045776367
102.62749862670898
102.64625091552735
102.65374908447265
102.6349967956543
102.63124771118164
102.61625137329102
102.62374954223633
102.61625137329102
102.60499725341796
102.59749908447266
102.58625183105468
102.6087532043457
102.6087532043457
102.59000091552734
102.56000137329102
102.54874725341797
102.50750045776367
102.48125
102.47750091552734
102.47750091552734
102.43249816894532
102.36500091552733
102.36874999999999
102.3762481689453
102.35750274658201
102.30125274658202
102.25999908447264
102.22999954223631
102.23749771118163
102.22999954223631
102.24124679565429
102.27500228881834


In [60]:
tempData.Count

11

In [61]:
using System;
using System.Drawing;
async Task readTempCondition(int count = 10, double maxTemp = 100){   
    using (var ledMatrix = new SenseHatLedMatrixI2c())
    using (var sh = new SenseHat())
    {
        ledMatrix.Fill(Color.Black);
        for (var i = 0; i < count; i++){
            var temp = sh.Temperature2.DegreesFahrenheit;
            Console.WriteLine(temp);
            if (temp > maxTemp) {
                ledMatrix.Fill(Color.Red);
            } 
            await Task.Delay(500);
        }
    }
}

### read temperature 50 times and add max temp condition

In [63]:
await readTempCondition(50, 102.5);

101.99375228881834
101.9824981689453
101.9900032043457
101.99375228881834
102.02000274658202
102.02375183105468
102.0387481689453
102.0387481689453
102.02750091552733
102.0387481689453
102.053751373291
102.07625274658201
102.09124908447265
102.08000183105467
102.09124908447265
102.0949981689453
102.1025032043457
102.0949981689453
102.09874725341795
102.110001373291
102.1025032043457
102.09874725341795
102.11375045776366
102.11375045776366
102.11375045776366
102.12124862670898
102.11375045776366
102.11749954223632
102.110001373291
102.11749954223632
102.13625183105468
102.13250274658202
102.13625183105468
102.13625183105468
102.14000091552734
102.14374999999998
102.14374999999998
102.14374999999998
102.12499771118163
102.12124862670898
102.12874679565428
102.12499771118163
102.12124862670898
102.09874725341795
102.11375045776366
102.09124908447265
102.07249679565429
102.06874771118163
102.06874771118163
102.08749999999999


### clear the ledMatrix to "black"

In [64]:
using (var ledMatrix = new SenseHatLedMatrixI2c())
{
        ledMatrix.Fill(Color.Black);
}


## @rondagdag

## https://www.linkedin.com/in/rondagdag